In [1]:
# 基本ライブラリ
import pandas as pd
import pandas.io.sql as psql
import numpy as np
import numpy.random as rd
import gc
import multiprocessing as mp
import os
import sys
import pickle
from collections import defaultdict
from glob import glob
import math
from datetime import datetime as dt
from pathlib import Path
import scipy.stats as st
import re
import shutil
from tqdm import tqdm_notebook as tqdm
import datetime
ts_conv = np.vectorize(datetime.datetime.fromtimestamp) # 秒ut(10桁) ⇒ 日付

# グラフ描画系
import matplotlib
from matplotlib import font_manager
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib import rc

from matplotlib import animation as ani
from IPython.display import Image

plt.rcParams["patch.force_edgecolor"] = True
#rc('text', usetex=True)
from IPython.display import display # Allows the use of display() for DataFrames
import seaborn as sns
sns.set(style="whitegrid", palette="muted", color_codes=True)
sns.set_style("whitegrid", {'grid.linestyle': '--'})
red = sns.xkcd_rgb["light red"]
green = sns.xkcd_rgb["medium green"]
blue = sns.xkcd_rgb["denim blue"]

#カラム内の文字数。デフォルトは50
pd.set_option("display.max_colwidth", 100)

#行数
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
#
pd.options.display.float_format = '{:,.5f}'.format

%matplotlib inline
%config InlineBackend.figure_format='retina'

In [2]:
DATA_VERSION = "v003"
TRIAL_NO = "093"
seed = 2069

In [3]:
path = f"../log/{DATA_VERSION}_{TRIAL_NO}/importance_fc_{DATA_VERSION}_{TRIAL_NO}_{seed}.csv"
importance_df = pd.read_csv(path, index_col=0)
importance_df = importance_df.pivot_table(values="importance", columns="fold", index="feature")
importance_df["ave"] = importance_df.mean(axis=1)
importance_df.sort_values("ave", inplace=True, ascending=False)
importance_df = importance_df.reset_index()

In [4]:
importance_df["ratio"] = importance_df["ave"] / importance_df["ave"].sum()

In [5]:
importance_df.to_excel(f"../log/{DATA_VERSION}_{TRIAL_NO}/importance_fc_all.xlsx")

In [6]:
importance_df["max"] = importance_df.iloc[:, :5].max(axis=1)

In [7]:
importance_df.sort_values("max", ascending=False)

fold,feature,1,2,3,4,5,ave,ratio,max
0,1J1st_AverageBondAngle,"10,127,935,611.26363","10,128,546,478.46309","10,092,101,273.27353","10,117,353,282.61989","10,064,038,541.19566","10,105,995,037.36316",0.91590,"10,128,546,478.46309"
1,dist_C_0_x,"279,505,476.87502","279,029,829.68422","279,139,796.09266","279,432,238.42380","278,922,846.01418","279,206,037.41798",0.02530,"279,505,476.87502"
2,eem2015ha_1,"98,970,265.10322","97,918,683.60356","100,903,396.79585","64,045,004.65914","100,969,724.32483","92,561,414.89732",0.00839,"100,903,396.79585"
4,molecule_type_dist_std_diff,"87,798,140.13422","62,904,457.22691","62,480,740.71265","66,616,321.19971","6,407,710.04247","57,241,473.86319",0.00519,"87,798,140.13422"
3,inv_distPR,"72,364,948.79101","62,345,161.20018","78,553,709.12838","74,291,936.23040","74,434,819.42264","72,398,114.95452",0.00656,"78,553,709.12838"
5,2Jd_idx1_2nd,"24,460,900.81395","50,182,326.91563","49,655,873.97539","47,073,350.98572","105,806,265.25481","55,435,743.58910",0.00502,"50,182,326.91563"
6,mmff94_0,"48,232,860.75935","49,037,350.36742","49,118,384.55565","49,003,086.84181","49,653,983.53859","49,009,133.21256",0.00444,"49,118,384.55565"
7,1Jlast_GetPartialCharge,"18,570,103.28582","10,734,010.39099","46,760,657.85921","20,200,088.00952","61,960,947.58012","31,645,161.42513",0.00287,"46,760,657.85921"
12,eem2015ba_1,"3,982,053.90081","5,512,307.91377","4,649,351.29248","38,814,866.10628","6,918,415.59982","11,975,398.96263",0.00109,"38,814,866.10628"
8,1J1st_AveSmallestBondAngle_diff,"32,914,264.39739","30,538,318.51914","31,457,962.51153","30,915,137.92492","28,030,733.66624","30,771,283.40384",0.00279,"32,914,264.39739"


In [9]:

qm9_cols = ['rc_A', 'rc_B', 'rc_C', 'mu', 'alpha',
       'homo', 'lumo', 'gap', 'r2', 'zpve', 'U0', 'U', 'H', 'G', 'Cv',
       'freqs_min', 'freqs_max', 'freqs_mean', 'linear', 'mulliken_min',
       'mulliken_max', 'mulliken_mean', 'mulliken_atom_0', 'mulliken_atom_1']
tmp = importance_df.sort_values("ratio", ascending=False)
tmp[tmp.feature.isin(qm9_cols)]

fold,feature,1,2,3,4,5,ave,ratio,max
24,mulliken_atom_0,"3,515,606.26718","3,472,943.95035","3,434,507.83088","3,675,282.74278","3,457,118.90372","3,511,091.93898",0.00032,"3,675,282.74278"
129,mulliken_atom_1,"240,661.28613","308,203.24116","325,769.79556","275,291.04842","285,669.29111","287,118.93247",0.00003,"325,769.79556"
176,gap,"157,010.14573","176,418.00965","132,151.27763","143,936.75272","176,316.89390","157,166.61592",0.00001,"176,418.00965"
217,lumo,"143,204.73590","110,740.26040","97,294.70134","84,006.06401","81,203.81863","103,289.91606",0.00001,"143,204.73590"
291,freqs_mean,"48,015.50117","29,241.50419","40,870.46331","50,503.43129","38,332.09814","41,392.59962",0.00000,"50,503.43129"
296,homo,"36,323.63716","40,326.37372","39,452.18974","44,078.13755","37,276.62515","39,491.39266",0.00000,"44,078.13755"
305,mu,"38,018.03603","40,023.67815","39,075.87113","28,793.20144","38,243.83842","36,830.92503",0.00000,"40,023.67815"
324,freqs_min,"26,684.44489","36,681.99492","27,528.53056","27,947.45349","27,935.68862","29,355.62250",0.00000,"36,681.99492"
331,rc_C,"21,237.76054","23,493.90985","26,389.43226","35,247.10554","23,605.92635","25,994.82691",0.00000,"35,247.10554"
336,rc_A,"26,034.73839","23,697.25358","25,557.27633","24,451.22167","25,464.30881","25,040.95976",0.00000,"26,034.73839"


In [10]:
for c in qm9_cols:
    print(c)

rc_A
rc_B
rc_C
mu
alpha
homo
lumo
gap
r2
zpve
U0
U
H
G
Cv
freqs_min
freqs_max
freqs_mean
linear
mulliken_min
mulliken_max
mulliken_mean
mulliken_atom_0
mulliken_atom_1


In [27]:
for c in importance_df[importance_df.ratio<0.000006].feature.values:
    print(c)

acsf_53_0
acsf_44_0
acsf_58_1
acsf_32_1
acsf_9_0
acsf_20_1
acsf_59_1
acsf_14_1
acsf_7_1
acsf_20_0
acsf_12_1
acsf_55_1
acsf_57_1
acsf_12_0
acsf_53_1
acsf_57_0
acsf_10_1
acsf_4_0
acsf_8_0
acsf_17_1
acsf_54_0
acsf_52_1
acsf_54_1
acsf_43_1
acsf_4_1
acsf_58_0
acsf_56_0
acsf_66_1
acsf_41_0
acsf_56_1
acsf_42_0
acsf_40_0
acsf_8_1
acsf_42_1
acsf_43_0
acsf_40_1
acsf_41_1
acsf_60_1
acsf_63_1
acsf_66_0
acsf_61_1
acsf_67_1
acsf_64_0
acsf_64_1
acsf_16_0
acsf_67_0
acsf_19_1
acsf_65_1
acsf_16_1
acsf_65_0
acsf_62_0
acsf_19_0
acsf_63_0
acsf_61_0
acsf_18_1
acsf_76_1
acsf_18_0
acsf_62_1
acsf_78_1
acsf_17_0
acsf_77_1
acsf_76_0
acsf_68_0
acsf_79_1
acsf_71_1
acsf_75_0
acsf_69_1
acsf_71_0
acsf_77_0
acsf_69_0
acsf_72_1
acsf_70_1
acsf_74_0
acsf_73_1
acsf_78_0
acsf_60_0
acsf_72_0
acsf_75_1
acsf_68_1
acsf_74_1
acsf_79_0
acsf_73_0
acsf_70_0


In [26]:
new_cols = [
'1J_ex1_dist_fromEx1_mean', '1J_ex1_dist_fromEx1_max',
'1J_ex1_dist_fromEx1_min', '1J_ex1_dist_fromEx1_std',
'1J_ex1_dist_fromEx1_max_min_diff', '1J_ex1_dist_from_first_mean',
'1J_ex1_dist_from_first_max', '1J_ex1_dist_from_first_min',
'1J_ex1_dist_from_first_std', '1J_ex1_dist_from_first_max_min_diff',
'1J_ex1_dist_from_last_mean', '1J_ex1_dist_from_last_max',
'1J_ex1_dist_from_last_min', '1J_ex1_dist_from_last_std',
'1J_ex1_dist_from_last_max_min_diff', '1J_ex1_angle_fromEx1_mean',
'1J_ex1_angle_fromEx1_max', '1J_ex1_angle_fromEx1_min',
'1J_ex1_angle_fromEx1_std', '1J_ex1_angle_fromEx1_max_min_diff',
'1J_ex1_torsion_F_L_EX1_mean', '1J_ex1_torsion_F_L_EX1_max',
'1J_ex1_torsion_F_L_EX1_min', '1J_ex1_torsion_F_L_EX1_std',
'1J_ex1_torsion_F_L_EX1_max_min_diff', '1J_ex1_cosT_F_L_EX1_mean',
'1J_ex1_cosT_F_L_EX1_max', '1J_ex1_cosT_F_L_EX1_min',
'1J_ex1_cosT_F_L_EX1_std', '1J_ex1_cosT_F_L_EX1_max_min_diff',
'1J_ex1_cos2T_F_L_EX1_mean', '1J_ex1_cos2T_F_L_EX1_max',
'1J_ex1_cos2T_F_L_EX1_min', '1J_ex1_cos2T_F_L_EX1_std',
'1J_ex1_cos2T_F_L_EX1_max_min_diff', '1J_ex1_Angle_0_1_mean',
'1J_ex1_Angle_0_1_max', '1J_ex1_Angle_0_1_min', '1J_ex1_Angle_0_1_std',
'1J_ex1_Angle_0_1_max_min_diff', '1J_ex1_dist_0_mean',
'1J_ex1_dist_0_max', '1J_ex1_dist_0_min', '1J_ex1_dist_0_std',
'1J_ex1_dist_0_max_min_diff', '1J_ex1_dist_1_mean', '1J_ex1_dist_1_max',
'1J_ex1_dist_1_min', '1J_ex1_dist_1_std', '1J_ex1_dist_1_max_min_diff',
'1J_ex1_GetHyb_mean', '1J_ex1_GetHyb_max', '1J_ex1_GetHyb_min',
'1J_ex1_GetHeteroValence_mean', '1J_ex1_GetHeteroValence_max',
'1J_ex1_GetHeteroValence_min', '1J_ex1_GetValence_mean',
'1J_ex1_GetValence_max', '1J_ex1_GetValence_min',
'1J_ex1_SpinMultiplicity_mean', '1J_ex1_SpinMultiplicity_max',
'1J_ex1_SpinMultiplicity_min', '1J_ex1_FormalCharge_mean',
'1J_ex1_FormalCharge_max', '1J_ex1_FormalCharge_min',
'1J_ex1_SmallestBondAngle_mean', '1J_ex1_SmallestBondAngle_max',
'1J_ex1_SmallestBondAngle_min', '1J_ex1_AverageBondAngle_mean',
'1J_ex1_AverageBondAngle_max', '1J_ex1_AverageBondAngle_min',
'1J_ex1_GetPartialCharge_mean', '1J_ex1_GetPartialCharge_max',
'1J_ex1_GetPartialCharge_min']

In [31]:
exist_col_imp = importance_df[~importance_df.feature.isin(new_cols)]
exist_col_imp

fold,feature,1,2,3,4,5,ave,ratio
0,1J1st_AverageBondAngle,"10,080,677,288.82402","10,090,126,617.65196","10,033,922,448.04514","10,073,079,506.20924","10,031,007,892.48314","10,061,762,750.64270",0.91199
1,dist_C_0_x,"279,514,583.18735","279,674,336.84655","278,600,590.18089","279,462,929.22700","278,542,620.54377","279,159,011.99711",0.02530
2,eem2015ha_1,"100,919,572.94378","100,619,237.02426","98,915,397.03644","67,416,031.57009","101,818,715.08395","93,937,790.73170",0.00851
3,1Jlast_GetPartialCharge,"74,267,532.39007","57,456,197.15193","113,497,034.97996","74,471,851.26933","115,722,871.41892","87,083,097.44204",0.00789
4,dist_to_type_mean,"69,255,281.12507","79,713,200.32747","76,917,648.11682","83,895,929.19298","88,895,341.02391","79,735,479.95725",0.00723
5,2Jd_idx1_2nd,"10,921,335.64564","62,987,181.45479","65,716,053.31873","90,081,471.56174","100,880,120.63340","66,117,232.52286",0.00599
6,molecule_type_dist_std_diff,"102,731,120.12066","48,789,766.38528","46,519,347.45391","22,545,016.68344","14,077,322.39353","46,932,514.60736",0.00425
7,mmff94_0,"44,281,206.03289","45,516,943.46718","45,174,921.52883","43,693,455.88701","44,283,574.55960","44,590,020.29510",0.00404
8,2J2nd_AverageBondAngle,"34,351,711.20969","32,642,501.08684","34,551,317.38892","34,188,705.72675","30,634,957.36530","33,273,838.55550",0.00302
9,1J1st_AveSmallestBondAngle_diff,"35,419,469.55182","34,115,235.80343","32,412,012.50054","34,026,480.31440","22,465,840.00008","31,687,807.63405",0.00287


In [34]:
for c in exist_col_imp[exist_col_imp.ratio < 0.00001].feature.tolist():
    print(c)

molecule_atom_index_0_dist_mean_diff
d_O_from2nd_ratio_0
molecule_atom_index_0_dist_mean
interBond_IsInRing
dist_N_1_x
qeq_0
molecule_atom_index_0_dist_std_div
3J2nd_AverageBondAngle
qtpie_0
3J3rd__C
molecule_dist_min
2Jdist_from2nd_max_mean_diff
a1_inring5
molecule_type_dist_mean_diff
interBond_IsRotor
mean_circle_size
n_circle
2Jdist_from2nd_mean
mean_dist_ratio_O_from2nd
1J1st_CountFreeOxygens
molecule_atom_index_1_dist_max_diff
interBond_BondOrder
max_circle_size
3J3rd_isChiral
a0_nb_inring4
1J1st_BOSum
a0_nb_inring3
1J1st_IsAromatic
molecule_atom_index_1_dist_mean_diff
3Jdist_from2nd_max_mean_diff
3J3rd__N
molecule_atom_index_1_dist_mean_div
pca_exp_1
pca_exp_2
2J2nd_isChiral
3Jlast_MemberOfRingSize
1Jlast_GetValence
3J3rd__O
a0_nb_nb_n
1J1st_IsInRing
min_circle_size
1Jlast_GetImplicitValence
3J2nd_isChiral
1Jlast_GetAtomicNum
interBond_IsAmider
3Jlast_isChiral
interBond_IsDoubleBondGeometry
interBond_IsClosure
1J1st_ExplicitHydrogenCount
3J3rd_isAroma
1J1st_IsChiral
1Jlast_GetHvy

In [27]:
new_col_imp = importance_df[importance_df.feature.isin(new_cols)]
new_col_imp

fold,feature,1,2,3,4,5,ave,ratio
13,1J_ex1_angle_fromEx1_mean,"15,111,072.89989","15,160,596.14558","3,965,521.84576","4,120,104.40444","3,609,662.12277","8,393,391.48369",0.00076
15,1J_ex1_angle_fromEx1_min,"3,403,609.19541","5,430,309.08313","7,698,493.20745","10,461,930.48694","8,709,251.39935","7,140,718.67446",0.00065
19,1J_ex1_angle_fromEx1_std,"5,517,333.85148","6,651,239.86708","7,154,443.71288","6,556,880.66427","5,862,694.55976","6,348,518.53110",0.00058
32,1J_ex1_Angle_0_1_max,"2,826,743.88346","1,107,374.79863","1,122,102.46134","3,099,219.60466","2,950,397.04705","2,221,167.55903",0.00020
34,1J_ex1_Angle_0_1_min,"506,144.45336","1,949,336.09338","2,805,986.25149","1,978,686.15403","2,931,667.08364","2,034,364.00718",0.00018
36,1J_ex1_dist_from_first_std,"1,186,764.81030","1,535,416.43849","1,560,882.80864","2,357,184.14031","1,941,765.80410","1,716,402.80037",0.00016
37,1J_ex1_dist_from_first_max_min_diff,"1,571,181.08230","1,677,041.04539","1,782,176.42848","1,550,144.39905","1,894,818.01591","1,695,072.19423",0.00015
40,1J_ex1_Angle_0_1_mean,"1,433,954.55384","1,747,373.74023","1,459,347.63854","1,465,032.09725","1,346,252.69904","1,490,392.14578",0.00014
45,1J_ex1_AverageBondAngle_max,"1,092,924.73508","1,172,050.73822","1,117,197.71854","1,397,787.42038","1,207,371.31198","1,197,466.38484",0.00011
46,1J_ex1_dist_1_std,"1,022,904.41386","1,233,790.79896","827,434.54661","1,435,485.96688","1,339,506.38830","1,171,824.42292",0.00011


In [29]:
new_col_imp[new_col_imp.ratio >= 0.00003].feature.tolist()

['1J_ex1_angle_fromEx1_mean',
 '1J_ex1_angle_fromEx1_min',
 '1J_ex1_angle_fromEx1_std',
 '1J_ex1_Angle_0_1_max',
 '1J_ex1_Angle_0_1_min',
 '1J_ex1_dist_from_first_std',
 '1J_ex1_dist_from_first_max_min_diff',
 '1J_ex1_Angle_0_1_mean',
 '1J_ex1_AverageBondAngle_max',
 '1J_ex1_dist_1_std',
 '1J_ex1_SmallestBondAngle_max',
 '1J_ex1_dist_1_min',
 '1J_ex1_dist_0_min',
 '1J_ex1_dist_0_std',
 '1J_ex1_dist_from_first_min',
 '1J_ex1_cos2T_F_L_EX1_mean',
 '1J_ex1_dist_from_first_max',
 '1J_ex1_dist_0_mean',
 '1J_ex1_dist_0_max_min_diff',
 '1J_ex1_cos2T_F_L_EX1_std',
 '1J_ex1_dist_1_max_min_diff',
 '1J_ex1_dist_from_first_mean',
 '1J_ex1_angle_fromEx1_max',
 '1J_ex1_dist_1_mean',
 '1J_ex1_Angle_0_1_std',
 '1J_ex1_angle_fromEx1_max_min_diff']

In [19]:
importance_df["ratio"] = importance_df["ave"]/importance_df["ave"].sum()

In [20]:
importance_df

fold,feature,1,2,3,4,5,ave,ratio
0,1J1st_AverageBondAngle,"10,115,884,918.28777","10,104,623,380.19084","10,077,264,638.47040","10,084,160,112.46832","10,080,560,787.03906","10,092,498,767.29128",0.91468
1,dist_C_0_x,"280,120,014.54616","279,833,531.23166","279,593,835.31282","279,821,082.72638","279,872,226.20915","279,848,138.00524",0.02536
2,molecule_type_dist_std_diff,"115,105,961.16502","114,715,012.15516","114,084,182.62603","114,797,520.95381","114,808,629.88491","114,702,261.35698",0.01040
3,eem2015ha_1,"102,088,081.00437","100,064,969.96234","100,210,704.75236","66,487,691.59089","99,173,690.76682","93,605,027.61536",0.00848
4,dist_to_type_mean,"81,244,121.25743","89,462,339.00229","77,507,890.13287","90,773,896.35203","90,054,689.16886","85,808,587.18270",0.00778
5,1Jlast_GetPartialCharge,"46,833,816.37383","47,060,510.64718","74,688,681.82621","67,855,604.92062","61,312,015.03210","59,550,125.75999",0.00540
6,mmff94_0,"45,796,584.16040","46,252,482.76216","46,347,976.04719","46,407,730.69059","43,156,997.36417","45,592,354.20490",0.00413
7,2J2nd_AverageBondAngle,"41,803,603.32371","41,356,164.24348","40,960,581.55232","41,677,370.80667","42,094,992.12510","41,578,542.41026",0.00377
8,1J1st_AveSmallestBondAngle_diff,"23,456,526.97180","33,096,529.84937","37,730,051.26893","25,035,991.40652","23,994,268.74588","28,662,673.64850",0.00260
9,cosT,"22,968,294.08123","22,759,577.35705","22,757,029.44653","22,737,032.08236","22,601,136.37542","22,764,613.86852",0.00206


In [22]:
importance_df_y = importance_df[importance_df.feature.isin(yiemon_HnJ)]

In [26]:
importance_df_y_high = importance_df_y[importance_df_y.ratio >= 0.00005]
importance_df_y_high

fold,feature,1,2,3,4,5,ave,ratio
13,mean_dist_C_from2nd,"2,219,107.80720","1,082,870.41410","12,345,291.11046","5,667,477.47080","12,604,486.16454","6,783,846.59342",0.00061
17,mean_angle_C_from2nd,"4,464,357.23324","4,365,691.19720","5,100,971.37583","4,677,631.60805","6,538,590.93950","5,029,448.47077",0.00046
36,d_O_from2nd_ratio_0,"1,745,468.21043","1,250,039.08453","1,385,857.75820","1,378,011.21881","1,703,789.89275","1,492,633.23294",0.00014
39,mean_dist_ratio_C_from2nd,"1,174,243.71498","1,314,657.43831","1,302,007.18286","1,324,328.27011","1,197,997.00485","1,262,646.72222",0.00011
40,mean_dist_ratio_O_from1st,"2,685,933.43542","241,324.03499","236,974.91415","2,122,986.04824","918,808.59357","1,241,205.40528",0.00011
41,mean_dist_ratio_O_from2nd,"548,789.52500","2,173,530.02903","1,247,912.36466","954,880.55809","1,167,358.27827","1,218,494.15101",0.00011
42,d_O_from1st_ratio_0,"227,104.04968","1,299,200.63347","2,450,670.81099","631,376.41198","1,346,113.18302","1,190,893.01783",0.00011
56,mean_angle_O_from2nd,"505,996.26947","495,110.14479","1,123,202.96260","889,573.24679","806,021.17185","763,980.75910",0.00007
65,d_O_from1st_0,"542,893.52662","407,577.05864","522,879.46225","709,298.34846","801,393.86190","596,808.45157",0.00005
67,mean_dist_C_from1st,"570,900.98960","577,426.49378","561,287.75217","666,204.12254","450,796.57512","565,323.18664",0.00005


In [28]:
importance_df_y_high.feature.tolist()


['mean_dist_C_from2nd',
 'mean_angle_C_from2nd',
 'd_O_from2nd_ratio_0',
 'mean_dist_ratio_C_from2nd',
 'mean_dist_ratio_O_from1st',
 'mean_dist_ratio_O_from2nd',
 'd_O_from1st_ratio_0',
 'mean_angle_O_from2nd',
 'd_O_from1st_0',
 'mean_dist_C_from1st']

In [18]:
for c in pd.read_csv("/Users/kenichi.matsui/Downloads/rdkit_train.csv").columns:
    print(c)

Unnamed: 0
id
a1_degree
a1_hybridization
a1_inring
a1_inring3
a1_inring4
a1_inring5
a1_inring6
a1_inring7
a1_inring8
a1_nb_h
a1_nb_o
a1_nb_c
a1_nb_n
a1_nb_na
a0_nb_degree
a0_nb_hybridization
a0_nb_inring
a0_nb_inring3
a0_nb_inring4
a0_nb_inring5
a0_nb_inring6
a0_nb_inring7
a0_nb_inring8
a0_nb_nb_h
a0_nb_nb_o
a0_nb_nb_c
a0_nb_nb_n
a0_nb_nb_na
x_a0_nb
y_a0_nb
z_a0_nb
a1_nb_degree
a1_nb_hybridization
a1_nb_inring
a1_nb_inring3
a1_nb_inring4
a1_nb_inring5
a1_nb_inring6
a1_nb_inring7
a1_nb_inring8
a1_nb_nb_h
a1_nb_nb_o
a1_nb_nb_c
a1_nb_nb_n
a1_nb_nb_na
x_a1_nb
y_a1_nb
z_a1_nb
dist_to_type_mean
